In [1]:
# import modules

import json
import os
import sys
import torch
import numpy as np
import tqdm
from scipy.special import softmax

from deepvisualinsight.MMS import MMS

In [ ]:
# record uncertainty score and diversity score

In [2]:
# Least Confidence
content_path = "E:\\DVI_exp_data\\active_learning\\baseline\\LeastConfidence\\resnet18\\CIFAR10"
sys.path.append(content_path)
from Model.model import *
net = ResNet18()

classes = ("airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")
mms = MMS(content_path, net, 1, 10, 1, 512, 10, classes, cmap="tab10", resolution=100, neurons=256, verbose=1, temporal=False, split=-1, advance_border_gen=True, attack_device="cuda:0")

for n_epoch in tqdm.tqdm(range(1,11,1)):
    state_dict = torch.load(os.path.join(mms.model_path, "Epoch_{}".format(n_epoch),"subject_model.pth"))
    mms.model.load_state_dict(state_dict)
    mms.model.to(mms.device)
    train_uncertainty = np.zeros(50000)
    batch_size = 200
    with torch.no_grad():
        for i in range(250):
            x = mms.training_data[i*batch_size:(i+1)*batch_size].to(mms.device)
            out = mms.model(x).cpu().numpy()
            out = softmax(out, axis=1)
            out = np.amax(out, axis=1)
            out = 1 - out
            train_uncertainty[i*batch_size:(i+1)*batch_size] = out

    f = open(os.path.join(mms.model_path, "Epoch_{}".format(n_epoch), "train_uncertainty_score.json"),"w")
    json.dump(train_uncertainty.tolist(), f)
    f = open(os.path.join(mms.model_path, "Epoch_{}".format(n_epoch), "train_total_score.json"),"w")
    json.dump(train_uncertainty.tolist(), f)

    test_uncertainty = np.zeros(10000)
    with torch.no_grad():
        for i in range(50):
            x = mms.testing_data[i*batch_size:(i+1)*batch_size].to(mms.device)
            out = mms.model(x).cpu().numpy()
            out = softmax(out, axis=1)
            out = np.amax(out, axis=1)
            out = 1 - out
            test_uncertainty[i*batch_size:(i+1)*batch_size] = out
    f = open(os.path.join(mms.model_path, "Epoch_{}".format(n_epoch), "test_uncertainty_score.json"),"w")
    json.dump(test_uncertainty.tolist(), f)
    f = open(os.path.join(mms.model_path, "Epoch_{}".format(n_epoch), "test_total_score.json"), "w")
    json.dump(test_uncertainty.tolist(), f)



  0%|          | 0/10 [00:00<?, ?it/s]

Finish loading content!


100%|██████████| 10/10 [03:22<00:00, 20.22s/it]


In [5]:
# LL4AL
content_path = "E:\\DVI_exp_data\\active_learning\\baseline\\LL4AL\\resnet18\\CIFAR10"
sys.path.append(content_path)
from Model.model import *
net = ResNet18()
lossnet = LossNet()

classes = ("airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")
mms = MMS(content_path, net, 1, 10, 1, 512, 10, classes, cmap="tab10", resolution=100, neurons=256, verbose=1, temporal=False, split=-1, advance_border_gen=True, attack_device="cuda:0")

for n_epoch in tqdm.tqdm(range(1,11,1)):
    state_dict = torch.load(os.path.join(mms.model_path, "Epoch_{}".format(n_epoch),"subject_model.pth"))
    mms.model.load_state_dict(state_dict)
    mms.model.to(mms.device)
    state_dict = torch.load(os.path.join(mms.model_path, "Epoch_{}".format(n_epoch),"lossPredNet.pth"))
    lossnet.load_state_dict(state_dict)
    lossnet.to(mms.device)
    train_uncertainty = np.zeros(50000)
    batch_size = 200
    with torch.no_grad():
        for i in range(250):
            x = mms.training_data[i*batch_size:(i+1)*batch_size].to(mms.device)
            _, features = mms.model.get_features(x)
            losses = lossnet(features).cpu().numpy().squeeze()
            train_uncertainty[i*batch_size:(i+1)*batch_size] = losses

    f = open(os.path.join(mms.model_path, "Epoch_{}".format(n_epoch), "train_uncertainty_score.json"),"w")
    json.dump(train_uncertainty.tolist(), f)
    f = open(os.path.join(mms.model_path, "Epoch_{}".format(n_epoch), "train_total_score.json"),"w")
    json.dump(train_uncertainty.tolist(), f)

    test_uncertainty = np.zeros(10000)
    with torch.no_grad():
        for i in range(50):
            x = mms.testing_data[i*batch_size:(i+1)*batch_size].to(mms.device)
            _, features = mms.model.get_features(x)
            losses = lossnet(features).cpu().numpy().squeeze()
            test_uncertainty[i*batch_size:(i+1)*batch_size] = losses
    f = open(os.path.join(mms.model_path, "Epoch_{}".format(n_epoch), "test_uncertainty_score.json"),"w")
    json.dump(test_uncertainty.tolist(), f)
    f = open(os.path.join(mms.model_path, "Epoch_{}".format(n_epoch), "test_total_score.json"), "w")
    json.dump(test_uncertainty.tolist(), f)

  0%|          | 0/10 [00:00<?, ?it/s]

Finish loading content!


100%|██████████| 10/10 [03:22<00:00, 20.23s/it]
